In [1]:
import pickle
from collections import OrderedDict, defaultdict
import operator
import numpy as np


In [59]:
def get_relevant_labels(doc):

    missing_labels = list(doc['missing_labels'].keys())
    wrong_labels = list(doc['wrong_labels'].keys())

    labels_items = {label: defaultdict(list) for label in list(set(missing_labels + wrong_labels))}

    for explainability_missing_dict in list(doc.values()):
        for one_label, missings_items in explainability_missing_dict.items():
            for one_entry in missings_items:
                items_dict = dict(one_entry["sentence"])
                for token, value in items_dict.items():
                    labels_items[one_label][token].append(value)

    for one_label in list(labels_items.keys()):
        for token, values in labels_items[one_label].items():
            labels_items[one_label][token] = np.mean(values)

        filtered_label_items = {
            item: score
            for item, score in labels_items[one_label].items()
            if abs(score) > 0.3
        }
        sorted_dict = dict(
            sorted(
                filtered_label_items.items(), key=operator.itemgetter(1), reverse=True
            )
        )
        pos_labels = [k for k, v in sorted_dict.items() if v>0]
        neg_labels = [k for k, v in sorted_dict.items() if v<0]
        
        labels_items[one_label] = {
            '+': pos_labels,
            '-': neg_labels
        }

    return labels_items

dict_labels = {}
for name in ["explainability_results_badly_performing_tags.pickle", "explainability_results_well_performing_tags.pickle"]:
    file = open(name, 'rb')
    explaiability_doct = pickle.load(file)

    dict_labels.update(get_relevant_labels(explaiability_doct))

dict_labels = OrderedDict(sorted(dict_labels.items()))

In [60]:
len(dict_labels)

31

In [61]:
for label, list_entries in dict_labels.items():
    print('->'.join(label.split('->')[1:]))
    print(list_entries)
    print('\n')

Education->Learning Environment
{'+': ['▁WhatsApp', '▁nutri', '▁learning', '▁birth', '▁educat', '▁educational', '▁educa', '▁schools', '▁illi', 'school', '▁virus', '▁school', '▁School', '▁Child', '▁Education', '▁cultural', '▁Language', '▁situat', '▁recover', '▁vert', '▁certificate', '▁psycho', '▁legisla', 'ification', '▁Learning', 'NA', '▁infants', '▁might', '▁Situation', '▁death', 'ection', '▁classes', '▁exam', '▁Argentina', '▁accesso', '▁garden', '▁education', '▁circulation', '▁ICT', '▁funding', 'toxic'], '-': ['▁traffic', '▁peace', 'ED', '▁sending', '▁Teacher', '▁tablets', '▁radio', '▁LGBT', '▁Safety', '▁mine', 'LP', '▁Traffic', '▁repres', '▁cards', '▁denuncia', '▁shooting', '▁includes']}


Education->Teachers and Education Personnel
{'+': ['▁pedagogi', '▁teachers', '▁military', '▁700', '▁teacher', '▁Student', '▁education', '▁didactic', '▁courses', '▁fire', '▁infants', '▁school', 'virus', '▁School', '▁Learning', '▁Education', '▁schools'], '-': ['▁amount', '▁documentation', '▁tui', '▁